## Getting Started with the Maxar Geospatial Platform Streaming API
This sample workflow will guide a user through using the Maxar Geospatial Platform streaming API with real world examples.
The process will go through:
* Searching for an image
* Selecting an image
* Downloading an image 
* Searching seamline imagery via basemaps

#### Authentication
For a more detailed view of how to authorize with MGP, please see the *Auth and Token Service* notebook in this collection

In [ ]:
import requests
import json

username = "your.email@address.com"
password= "yourpassword"

url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Search for an image
The following request will search for imagery on the below bounding box (bbox). The return will be a dictionary containing metadata and results for the specified search.

In [ ]:
bbox = "39.710884,-105.243530,39.808536,-105.117874"
srs = "EPSG:4326"
height = 256
width = 256
output = "application/json"
type_name = "Maxar:FinishedFeature"

search_url = "https://api.maxar.com/streaming/v1/ogc/wfs?service=WFS&request=GetFeature&version=2.0.0&bbox={}&" \
             "srsName={}&typeNames={}&height={}&width={}&" \
             "outputFormat={}".format(bbox, srs, type_name, height, width, output)

search_response = requests.request("GET", search_url, headers=access_token)

print(search_response.json())

#### Select an Image
Next a feature will be isolated from the list of features from the results above. Below, features will be a list of dictionaries where each dictionary in the list is a unique feature.

In [ ]:
WV02_features = [i for i in search_response.json()['features'] if i['properties']['source'] == 'WV02']
print(WV02_features)

Now an image will be selected from the specified search. The first image in the `WV02_features` list will be chosen for this example. Because a feature profile was not selected, the default profile will return features based on age. The first image will then be the newest image.

In [ ]:
desired_image = WV02_features[0]
desired_featureid = desired_image['id']
print(desired_image)
print(desired_featureid)

This image can now be viewed utilizing a WMS call. The IPython module will need to be imported in order to have the ability to display images.

In [ ]:
wms_url = "https://api.maxar.com/streaming/v1/ogc/wms?service=WMS&request=GetMap&version=1.3.0&bbox={}&crs={}&" \
          "layers=Maxar:Imagery&height={}&width={}&format=image/vnd.jpeg-png&transparent=true&tiled=true&" \
          "cql_Filter=featureId='{}'".format(bbox, srs, height, width, desired_featureid)

wms_response = requests.request("GET", wms_url, headers=access_token)

import IPython.display as disp
from IPython.display import Image, display
display(disp.Image(wms_response.content))

A tile view of this image in full resolution can also be viewed utilizing a WMTS request.

In [ ]:
wmts_url = "https://api.maxar.com/streaming/v1/ogc/gwc/service/wmts?service=WMTS&request=GetTile&version=1.0.0&" \
           "layer=Maxar:Imagery&tileMatrixSet={}&tileMatrix=EPSG:4326:18&tileCol=108959&tileRow=73232&format=image/png&" \
           "style=raster".format(srs)

wmts_response = requests.request("GET", wmts_url, headers=access_token)

display(disp.Image(wmts_response.content))

#### Download an Image
Now that the desired feature has been isolated to download, a download location will need to specified.
*Note: This sets the output location to the current working directory.*

In [ ]:
import os
output_location = os.path.join(os.getcwd(), 'my_image.jpeg')
print(output_location)

The image can now be downloaded as a jpeg.

In [ ]:
with open(output_location, 'wb') as output_file:
    output_file.write(wms_response.content)
print("File successfully downloaded to {}".format(os.getcwd()))

#### Basemaps

#### Search for a seamlines feature
The following request will search for imagery on the below Bbox. The return will be a dictionary containing metadata and results for the desired search.

In [ ]:
bbox = "39.746196,-104.954672,39.747858,-104.951625"
typename = "seamline"
output = "json"
srs = "EPSG:4326"
cql_filter = "BBOX(seamline_geometry,{})AND(product_name='VIVID_STANDARD_30')".format(bbox)

seamline_search_url = "https://api.maxar.com/basemaps/v1/seamlines/wfs?service=WFS&request=GetFeature&version=2.0.0&" \
                      "typename={}&outputFormat={}&srsName={}&cql_filter={}".format(typename, output, srs, cql_filter)

seamline_search_response = requests.request("GET", seamline_search_url, headers=access_token)

print(seamline_search_response.json())

#### Show seamlines for a given AOI
Using the WMS method, a user can obtain seamlines over a desired AOI.

In [ ]:
bbox = "36.985003,-109.028320,40.996484,-102.041016"
cql_filter = "product_name='VIVID_STANDARD_30'"

seamline_wms_url = "https://api.maxar.com/basemaps/v1/seamlines/ows?SERVICE=WMS&REQUEST=GetMap&VERSION=1.3.0&BBOX={}" \
                   "&CRS={}&LAYERS=Maxar:seamline&HEIGHT=300&WIDTH=300&FORMAT=image/jpeg&TRANSPARENT=True&" \
                   "TILED=true&cql_filter={}".format(bbox, srs, cql_filter)

seamline_wms_response = requests.request("GET", seamline_wms_url, headers=access_token)

display(disp.Image(seamline_wms_response.content))

A user can also obtain individual seamline tiles via the WMTS method. A larger bbox will return more tiles.

In [ ]:
row = 4574
col = 6831

seamline_wmts_url = "https://api.maxar.com/basemaps/v1/seamlines/gwc/service/wmts?service=WMTS&request=GetTile&" \
                    "version=1.0.0&layer=Maxar:seamline&format=image/jpeg&tileMatrixSet={}&tileMatrix=EPSG:4326:14&" \
                    "tileCol={}&tileRow={}&cql_filter={}".format(srs, col, row, cql_filter)

seamline_wmts_response = requests.request("GET", seamline_wmts_url, headers=access_token)

display(disp.Image(seamline_wmts_response.content))